# Pommerman Demo.

This notebook demonstrates how to train Pommerman agents. Please let us know at support@pommerman.com if you run into any issues.

In [1]:
import os
import sys
import numpy as np
import time

import pommerman
from pommerman.agents import SimpleAgent, RandomAgent, PlayerAgent, BaseAgent
from pommerman.configs import ffa_v0_fast_env
from pommerman.envs.v0 import Pomme as Pomme_v0
from pommerman.characters import Bomber
from pommerman import utility
from pommerman import agents
from pommerman import envs
from pommerman import constants
from pommerman import characters
from pommerman import configs

# print all env configs
print(pommerman.REGISTRY)

['AdvancedLesson-v0', 'PommeFFACompetition-v0', 'PommeFFACompetitionFast-v0', 'PommeFFAFast-v0', 'PommeFFA-v1', 'PommeFFAFast-v3', 'PommeFFAFast-v4', 'Lesson1-v0', 'Lesson2-v0', 'Lesson2b-v0', 'Lesson2c-v0', 'Lesson2d-v0', 'Lesson2e-v0', 'Lesson3-v0', 'Lesson3b-v0', 'Lesson3c-v0', 'Lesson3d-v0', 'OneVsOne-v0', 'PommeRadioCompetition-v2', 'PommeRadio-v2', 'Simple-v0', 'SimpleRandomTeam-v0', 'SimpleTeam-v0', 'PommeTeamCompetition-v0', 'PommeTeamCompetitionFast-v0', 'PommeTeamCompetition-v1', 'PommeTeam-v0', 'PommeTeamFast-v0', 'PommeTeamSimple-v0']


# Train with stable baseline

In [2]:
import gym
from gym import spaces
import tensorflow as tf
from stable_baselines.a2c.utils import linear
from stable_baselines.common.policies import ActorCriticPolicy, MlpPolicy, CnnPolicy, FeedForwardPolicy
from stable_baselines.common.vec_env import SubprocVecEnv, DummyVecEnv
from stable_baselines import PPO2

## Inherit pommerman env and make it compatible with stable-baseline

In [3]:
class CustomPomme(Pomme_v0):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.obs_raw = None # store the raw version of observation
        self.training_idx = 1 # idx of the agent being trained
    
    # function to flatten pommerman observation
    def _transform_obs(self, obs_raw):
        obs_training = obs_raw[self.training_idx] # default the first agent to be trained

        # construct flattened observation
        obs = [
            *np.array(obs_training["board"]).reshape(-1),
            *np.array(obs_training["bomb_blast_strength"]).reshape(-1),
            *np.array(obs_training["bomb_life"]).reshape(-1),
            *np.array(obs_training["position"]).reshape(-1),
            obs_training["ammo"],
            obs_training["blast_strength"],
            obs_training["can_kick"],
            obs_training["teammate"].value,
            obs_training["enemies"][0].value,
            
            # uncommon if training 1 v 1
            obs_training["enemies"][0].value,
            obs_training["enemies"][0].value,
            
            # uncommon if training 2 v 2
#             obs_training["enemies"][1].value,
#             obs_training["enemies"][2].value,
        ]
        return obs
    
    def get_obs_raw(self):
        return self.obs_raw

    def step(self, action_training):
        action_nontraining = self.act(self.obs_raw)
        actions = [*action_nontraining, action_training]
        obs_raw, reward, done, info = super().step(actions)
        self.obs_raw = obs_raw
        return self._transform_obs(obs_raw), reward[self.training_idx], done, info
    
    def reset(self):
        obs_raw = super().reset()
        self.obs_raw = obs_raw
        return self._transform_obs(obs_raw)
    
    def render(self,
               mode=None,
               close=False,
               record_pngs_dir=None,
               record_json_dir=None,
               do_sleep=True):
        super().render(mode=mode,
                       close=close,
                       record_pngs_dir=record_pngs_dir,
                       record_json_dir=record_json_dir,
                       do_sleep=do_sleep)

In [4]:
class CustomCNN(ActorCriticPolicy):
    def __init__(self, sess, ob_space, ac_space, n_env, n_steps, n_batch, reuse=False, **kwargs):
        super(CustomCNN, self).__init__(sess, ob_space, ac_space, n_env, n_steps, n_batch, reuse=reuse)
        ob_len = ob_space.shape[0]
        size=11
        bp = 3*size**2 #board partition
        with tf.variable_scope("model", reuse=reuse):
            obs = self.processed_obs
            #print("Okay, what's going on?", obs.shape)
            self.board1, self.misc = tf.split(obs, [bp, -1], 1)
            #print("Initial shapes:", self.board1.shape, self.misc.shape)
            
            self.board = tf.reshape(self.board1, (-1, size, size, 3))
            #print("Processed shapes:", self.board.shape, self.misc.shape)
            self.conv1 = tf.layers.conv2d(self.board, 64, 2, activation=tf.nn.relu, name='conv1')
            self.conv2 = tf.layers.conv2d(self.conv1, 32, 2, activation=tf.nn.relu, name='conv2')
            self.fc0 = tf.contrib.layers.flatten(self.conv2)
            #print("fc shapes:", self.fc0.shape, self.misc.shape)
            self.fc1 = tf.concat((self.fc0, self.misc), -1)
            #print("Catted shape", self.fc1.shape)
            self.fc1 = tf.layers.dense(self.fc1, 1024, name = 'fc1')
            self.actions = tf.layers.dense(self.fc1, 6)   
            self.valueUM = tf.layers.dense(self.fc1, 128) #??

            self._proba_distribution, self._policy, self.q_value = \
                self.pdtype.proba_distribution_from_latent(self.actions, self.valueUM, init_scale=0.01)

        self._value_fn = linear(self.valueUM, 'vf', 1)
        self._setup_init()

    def step(self, obs, state=None, mask=None, deterministic=False):
        #print(obs)
        #b, c, conv, flat = self.sess.run((self.board, self.conv1, self.conv2, self.fc0), {self.obs_ph:obs})
        #print(b.shape, c.shape, conv.shape)
        if deterministic:
            action, value, neglogp = self.sess.run([self.deterministic_action, self.value_flat, self.neglogp],
                                                   {self.obs_ph: obs})
        else:
            action, value, neglogp = self.sess.run([self.action, self.value_flat, self.neglogp],
                                                   {self.obs_ph: obs})
        return action, value, self.initial_state, neglogp

    def proba_step(self, obs, state=None, mask=None):
        return self.sess.run(self.policy_proba, {self.obs_ph: obs})

    def value(self, obs, state=None, mask=None):
        return self.sess.run(self.value_flat, {self.obs_ph: obs})

In [5]:
class CustomCNN2(ActorCriticPolicy):
    def __init__(self, sess, ob_space, ac_space, n_env, n_steps, n_batch, reuse=False, **kwargs):
        super(CustomCNN2, self).__init__(sess, ob_space, ac_space, n_env, n_steps, n_batch, reuse=reuse)
        ob_len = ob_space.shape[0]
        size=11
        bp = 3*size**2 #board partition
        with tf.variable_scope("model", reuse=reuse):
            obs = self.processed_obs
            self.board1, self.misc = tf.split(obs, [bp, -1], 1)
            
            self.board = tf.reshape(self.board1, (-1, size, size, 3))
            self.conv1 = tf.layers.conv2d(self.board, 64, 3, activation=tf.nn.relu, name='conv1')
            self.conv2 = tf.layers.conv2d(self.conv1, 64, 3, activation=tf.nn.relu, name='conv2')
            self.conv3 = tf.layers.conv2d(self.conv2, 64, 3, activation=tf.nn.relu, name='conv3')
            self.conv4 = tf.layers.conv2d(self.conv3, 64, 3, activation=tf.nn.relu, name='conv4')
            self.conv5 = tf.layers.conv2d(self.conv4, 64, 3, activation=tf.nn.relu, name='conv5')
            self.fc0 = tf.contrib.layers.flatten(self.conv5)
            self.fc0 = tf.concat((self.fc0, self.misc), -1)
            self.fc1 = tf.layers.dense(self.fc0, 1024, name = 'fc1')
            self.fc2 = tf.layers.dense(self.fc1, 200, name = 'fc2')
            self.fc3 = tf.layers.dense(self.fc2, 50, name = 'fc3')
            self.actions = tf.layers.dense(self.fc3, 6)   
            self.valueUM = tf.layers.dense(self.fc3, 128) #??

            self._proba_distribution, self._policy, self.q_value = \
                self.pdtype.proba_distribution_from_latent(self.actions, self.valueUM, init_scale=0.01)

        self._value_fn = linear(self.valueUM, 'vf', 1)
        self._setup_init()

    def step(self, obs, state=None, mask=None, deterministic=False):
        if deterministic:
            action, value, neglogp = self.sess.run([self.deterministic_action, self.value_flat, self.neglogp],
                                                   {self.obs_ph: obs})
        else:
            action, value, neglogp = self.sess.run([self.action, self.value_flat, self.neglogp],
                                                   {self.obs_ph: obs})
        return action, value, self.initial_state, neglogp

    def proba_step(self, obs, state=None, mask=None):
        return self.sess.run(self.policy_proba, {self.obs_ph: obs})

    def value(self, obs, state=None, mask=None):
        return self.sess.run(self.value_flat, {self.obs_ph: obs})

In [6]:
def team_v3_fast_env():
    """Start up a FFA config with the default settings."""
    env = CustomPomme2
    game_type = constants.GameType.Team
    env_entry_point = 'CustomPomme'
    env_id = 'PommeTeamFast-v3'
    env_kwargs = {
        'game_type': game_type,
        'board_size': 6,
        'num_rigid': 0,
        'num_wood': 0,
        'num_items': 0,
        'max_steps': constants.MAX_STEPS,
        'render_fps': 1000,
        'env': env_entry_point,
    }
    agent = characters.Bomber
    return locals()

def one_vs_one_v3_env():
    """Start up a FFA config with the default settings."""
    env = CustomPomme2
    game_type = constants.GameType.OneVsOne
    env_entry_point = 'CustomPomme'
    env_id = 'PommeOneVsOneFast-v3'
    env_kwargs = {
        'game_type': game_type,
        'board_size': 11,
        'num_rigid': 0,
        'num_wood': 0,
        'num_items': 0,
        'max_steps': constants.MAX_STEPS,
        'render_fps': 1000,
        'env': env_entry_point,
    }
    agent = characters.Bomber
    return locals()

def one_vs_one_v3_wood(wood):
    """Start up a FFA config with the default settings."""
    env = CustomPomme2
    game_type = constants.GameType.OneVsOne
    env_entry_point = 'CustomPomme'
    env_id = 'PommeOneVsOneFast-v3'
    env_kwargs = {
        'game_type': game_type,
        'board_size': 6,
        'num_rigid': 0,
        'num_wood': wood,
        'num_items': 0,
        'max_steps': constants.MAX_STEPS,
        'render_fps': 1000,
        'env': env_entry_point,
    }
    agent = characters.Bomber
    return locals()


def one_vs_one_v3_wood_walls(n):
    """Start up a FFA config with the default settings."""
    env = CustomPomme2
    game_type = constants.GameType.OneVsOne
    env_entry_point = 'CustomPomme'
    env_id = 'PommeOneVsOneFast-v3'
    env_kwargs = {
        'game_type': game_type,
        'board_size': 6,
        'num_rigid': n,
        'num_wood': n,
        'num_items': 0,
        'max_steps': constants.MAX_STEPS,
        'render_fps': 1000,
        'env': env_entry_point,
    }
    agent = characters.Bomber
    return locals()

In [7]:
class StaticAgent(BaseAgent):
    def act(self, obs, action_space):
        return 0

In [8]:
class RandomAgentNoBomb(RandomAgent):
    def act(self, obs, action_space):
        action = super().act(obs, action_space)
        if action == 5:
            action = 0
        return action

In [10]:
# log function during training, implement if needed
def log(local_var, global_var):
    pass
#     display(local_var)
#     display(global_var)

In [11]:
def train(env, path, timesteps):
    n_cpu = 1
    #env = DummyVecEnv([lambda: env_pom for i in range(n_cpu)])

    model = PPO2(CustomCNN, env, verbose=1, 
                 n_steps = 3000, # batch_size = n_step * num_env
                 ent_coef = 0.001, # entropy coefficient
                 tensorboard_log="./ppo_pommerman_tensorboard/")
    try:
        model.load(path)
    except ValueError: 
        pass
    model = model.learn(total_timesteps=timesteps, # num_update = total_timesteps // batch_size
                        callback = log)
    model.save(path)
    return model
    
#render: 0=no, 1=first game, 2=all
def validate(env, path, total, render=0):
    # del model # remove to demonstrate saving and loading
    model = PPO2.load(path)

    n_cpu = 1
    #env = DummyVecEnv([lambda: env_pom for i in range(n_cpu)])
    model.envs = env

    # test the learned model
    num_win = 0
    num_tie = 0
    num_lose = 0
    for i_episode in range(total):
        obs = env.reset()
        done = False
        info = None
        while not done:
            if render == 2 or (render == 1 and i_episode==0): env.render()
                
            action_training, _states = model.predict(obs)
    #         print(action_training)
            obs, rewards, dones, infos = env.step(action_training)
    #         print(infos)
            done = dones[0]
            info = infos[0]
            time.sleep(0.1)
        print('Episode {} finished'.format(i_episode))
        if(info["result"].value == 0):
            if(1 in info["winners"]):
                num_win+=1
            else:
                num_lose+=1
        elif(info["result"].value == 2):
            num_tie+=1
    #     print(info)
    env.close()
    print("Win ", num_win, "/", total, " games")
    print("Tie ", num_tie, "/", total, " games")
    print("Lose ", num_lose, "/", total, " games")
    return num_win, num_tie, num_lose

In [12]:
def define_env(config, agent, n_cpu=1):
    env_pom = CustomPomme(**config["env_kwargs"])

    # config agents
    agents = []

    # Add simple agents
    for agent_id in range(1):
        if agent == 'static':
            agents.append(StaticAgent(config["agent"](agent_id, config["game_type"])))
        elif agent == 'random':
            agents.append(RandomAgentNoBomb(config["agent"](agent_id, config["game_type"])))
        elif agent == 'simple':
            agents.append(SimpleAgent(config["agent"](agent_id, config["game_type"])))

    # add player agent(to train)
    agents.append(PlayerAgent(config["agent"](1, config["game_type"])))

    env_pom.set_agents(agents)
    env_pom.set_training_agent(agents[1].agent_id)
    env_pom.set_init_game_state(None)

    # Seed and reset the environment
    env_pom.seed(0)

    env = DummyVecEnv([lambda: env_pom for i in range(n_cpu)])
    return env

In [13]:
def lesson1_env():
    """Lesson 1-blank training config."""
    env = envs.v0.Pomme
    game_type = constants.GameType.OneVsOne
    env_entry_point = 'pommerman.envs.v0:Pomme'
    env_id = 'Lesson1-v0'
    env_kwargs = {
        'game_type': game_type,
        'board_size': 11,
        'num_rigid': 0,
        'num_wood': 0,
        'num_items': 0,
        'max_steps': 200,
        'render_fps': constants.RENDER_FPS,
        'rand_agent_pos': False,
        'env': env_entry_point,
    }
    agent = characters.Bomber
    return locals()

def lesson2_env():
    """Lesson 2-box training config."""
    env = envs.v0.Pomme
    game_type = constants.GameType.OneVsOne
    env_entry_point = 'pommerman.envs.v0:Pomme'
    env_id = 'Lesson2-v0'
    env_kwargs = {
        'game_type': game_type,
        'board_size': 11,
        'num_rigid': 0,
        'num_wood': 4,
        'num_items': 0,
        'max_steps': 200,
        'render_fps': constants.RENDER_FPS,
        'rand_agent_pos': False,
        'env': env_entry_point,
    }
    agent = characters.Bomber
    return locals()

def lesson2b_env():
    """Lesson 2b-box training config."""
    env = envs.v0.Pomme
    game_type = constants.GameType.OneVsOne
    env_entry_point = 'pommerman.envs.v0:Pomme'
    env_id = 'Lesson2b-v0'
    env_kwargs = {
        'game_type': game_type,
        'board_size': 11,
        'num_rigid': 0,
        'num_wood': 8,
        'num_items': 0,
        'max_steps': 200,
        'render_fps': constants.RENDER_FPS,
        'rand_agent_pos': False,
        'env': env_entry_point,
    }
    agent = characters.Bomber
    return locals()

def lesson2c_env():
    """Lesson 2c-box training config."""
    env = envs.v0.Pomme
    game_type = constants.GameType.OneVsOne
    env_entry_point = 'pommerman.envs.v0:Pomme'
    env_id = 'Lesson2c-v0'
    env_kwargs = {
        'game_type': game_type,
        'board_size': 11,
        'num_rigid': 0,
        'num_wood': 16,
        'num_items': 0,
        'max_steps': 300, #increasing timesteps to reduce reward sparsity per more challenging game
        'render_fps': constants.RENDER_FPS,
        'rand_agent_pos': False,
        'env': env_entry_point,
    }
    agent = characters.Bomber
    return locals()

def lesson2d_env():
    """Lesson 2d-box training config."""
    env = envs.v0.Pomme
    game_type = constants.GameType.OneVsOne
    env_entry_point = 'pommerman.envs.v0:Pomme'
    env_id = 'Lesson2d-v0'
    env_kwargs = {
        'game_type': game_type,
        'board_size': 11,
        'num_rigid': 0,
        'num_wood': constants.NUM_WOOD,
        'num_items': 0,
        'max_steps': 300,
        'render_fps': constants.RENDER_FPS,
        'rand_agent_pos': False,
        'env': env_entry_point,
    }
    agent = characters.Bomber
    return locals()

def lesson2e_env():
    """Lesson 2e-box training config."""
    env = envs.v0.Pomme
    game_type = constants.GameType.OneVsOne
    env_entry_point = 'pommerman.envs.v0:Pomme'
    env_id = 'Lesson2e-v0'
    env_kwargs = {
        'game_type': game_type,
        'board_size': 11,
        'num_rigid': 0,
        'num_wood': 72,
        'num_items': 0,
        'max_steps': 300,
        'render_fps': constants.RENDER_FPS,
        'rand_agent_pos': False,
        'env': env_entry_point,
    }
    agent = characters.Bomber
    return locals()

def lesson3_env():
    """Lesson 3-rigid training config."""
    env = envs.v0.Pomme
    game_type = constants.GameType.OneVsOne
    env_entry_point = 'pommerman.envs.v0:Pomme'
    env_id = 'Lesson3-v0'
    env_kwargs = {
        'game_type': game_type,
        'board_size': 11,
        'num_rigid': 4,
        'num_wood': constants.NUM_WOOD,
        'num_items': 0,
        'max_steps': 300,
        'render_fps': constants.RENDER_FPS,
        'rand_agent_pos': False,
        'env': env_entry_point,
    }
    agent = characters.Bomber
    return locals()

def lesson3b_env():
    """Lesson 3b-rigid training config."""
    env = envs.v0.Pomme
    game_type = constants.GameType.OneVsOne
    env_entry_point = 'pommerman.envs.v0:Pomme'
    env_id = 'Lesson3b-v0'
    env_kwargs = {
        'game_type': game_type,
        'board_size': 11,
        'num_rigid': 8,
        'num_wood': constants.NUM_WOOD,
        'num_items': 0,
        'max_steps': 300,
        'render_fps': constants.RENDER_FPS,
        'rand_agent_pos': False,
        'env': env_entry_point,
    }
    agent = characters.Bomber
    return locals()

def lesson3c_env():
    """Lesson 3c-rigid training config."""
    env = envs.v0.Pomme
    game_type = constants.GameType.OneVsOne
    env_entry_point = 'pommerman.envs.v0:Pomme'
    env_id = 'Lesson3c-v0'
    env_kwargs = {
        'game_type': game_type,
        'board_size': 11,
        'num_rigid': 16,
        'num_wood': constants.NUM_WOOD,
        'num_items': 0,
        'max_steps': 400, #increasing timesteps due to increasingly difficult environment
        'render_fps': constants.RENDER_FPS,
        'rand_agent_pos': False,
        'env': env_entry_point,
    }
    agent = characters.Bomber
    return locals()

def lesson3d_env():
    """Lesson 3d-rigid training config. Also used for lessons 4-7."""
    env = envs.v0.Pomme
    game_type = constants.GameType.OneVsOne
    env_entry_point = 'pommerman.envs.v0:Pomme'
    env_id = 'Lesson3d-v0'
    env_kwargs = {
        'game_type': game_type,
        'board_size': 11,
        'num_rigid': constants.NUM_RIGID,
        'num_wood': constants.NUM_WOOD,
        'num_items': 0,
        'max_steps': 400,
        'render_fps': constants.RENDER_FPS,
        'rand_agent_pos': False,
        'env': env_entry_point,
    }
    agent = characters.Bomber
    return locals()

def advanced_lesson_env():
    """Lesson 4+ training environment. Full number of timesteps"""
    env = envs.v0.Pomme
    game_type = constants.GameType.OneVsOne
    env_entry_point = 'pommerman.envs.v0:Pomme'
    env_id = 'AdvancedLesson-v0'
    env_kwargs = {
        'game_type': game_type,
        'board_size': 11,
        'num_rigid': constants.NUM_RIGID,
        'num_wood': constants.NUM_WOOD,
        'num_items': 0,
        'max_steps': constants.MAX_STEPS,
        'render_fps': constants.RENDER_FPS,
        'rand_agent_pos': False,
        'env': env_entry_point,
    }
    agent = characters.Bomber
    return locals()

In [14]:
conf1 = lesson1_env()
conf2 = lesson2_env()
conf3 = lesson2b_env()
conf4 = lesson2c_env()
conf5 = lesson2d_env()
conf6 = lesson2e_env()
conf7 = lesson3_env()
conf8 = lesson3b_env()
conf9 = lesson3c_env()
conf10 = lesson3d_env()
conf11 = advanced_lesson_env()

env1 = define_env(conf1, 'static')
env2 = define_env(conf2, 'static')
env3 = define_env(conf3, 'static')
env4 = define_env(conf4, 'static')
env5 = define_env(conf5, 'static')
env6 = define_env(conf6, 'static')
env7 = define_env(conf7, 'static')
env8 = define_env(conf8, 'static')
env9 = define_env(conf9, 'static')
env10 = define_env(conf10, 'static')
env11 = define_env(conf11, 'random')

env_list = [env1, env2, env3, env4, env5, env6, env7, env8, env9, env10, env11]

env_names = ['lesson1',
             'lesson2',
             'lesson2b',
             'lesson2c',
             'lesson2d',
             'lesson2e', 
             'lesson3',
             'lesson3b',
             'lesson3c',
             'lesson3d',
             'lesson4']

In [15]:
#No tansfers to establish a baseline
validate_num = 20

history = []
first = True

for i in range(len(env_list)):
    
    env = env_list[i]
    path = 'models/'+env_names[i]
    wins = []
    draws = []
    losses = []
    win = 0
    #if (not first):
    #    model.save(path)
    j=0
    while win < validate_num * .95:
        j+=1
        start = time.time()
        record = open('log.txt', 'a')
        print("Training", path)
        
        model = train(env, path, 500000)
        win, draw, loss = validate(env, path, validate_num)
        
        record.write(path +": "+ str(win) + " " + str(draw) +  " " + str(loss) + '\n')
        wins.append(win)
        draws.append(draw)
        losses.append(loss)
        record.close()
        print("1 epoch training time:", time.time() - start)
    history.append((wins, draws, losses))
    first = False




Training models/lesson1




Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



--------------------------------------
| approxkl           | 2.437245e-05  |
| clipfrac           | 0.0           |
| explained_variance | 0.00384       |
| fps                | 329           |
| n_updates          | 1             |
| policy_entropy     | 1.7917286     |
| policy_loss        | -0.0006425314 |
| serial_timesteps   | 3000          |
| time_elapsed       | 1.91e-06      |
| total_timesteps    | 3000          |
| value_loss         | 0.41281912    |
--------------------------------------


KeyboardInterrupt: 

In [16]:
#Loading the model
path = 'lesson2c'
model = PPO2(CustomCNN, env, verbose=1, 
             n_steps = 3000, # batch_size = n_step * num_env
             ent_coef = 0.001, # entropy coefficient
             tensorboard_log="./ppo_pommerman_tensorboard/")
model.load(path)


Loading a model without an environment, this model cannot be trained until it has a valid environment.
